In [ ]:
# Week 2 task - Data Wrangling
### Loading dataset 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
df = pd.read_csv("Online Retail_csv.csv")
df
# Identifying missing values
#identifying cols with missing vals
print("No of missing Values in columns: \n", df.isnull().sum())
missing_vals_cust = df["CustomerID"].isnull().sum() #sum of missing vals in customerID
missing_vals_descrp = df["Description"].isnull().sum() #sum of missing vals in description column
total = df["CustomerID"].shape[0]
total
print(f"Percentage of missing values in CustomerID:{round(missing_vals_cust/total * 100)}")
print(f"Percentage of missing values in Description:{missing_vals_descrp/total}")
# Handling missing values
#Number of missing vals in Descrp col is very small, almost insignificant, hence the strategic approach will be to delete thee column
df1 = df.dropna(subset=["Description"])
df1
#The IDs in the customer data are not unique, they are repeated for some customers. 
df2 = df1.fillna(method="bfill") #filled the null values with the next CustomerID
df2.isnull().sum()
## Checking for duplicate values
dup = df2[df2.duplicated()]
dup
no_dup = df2.drop_duplicates(inplace=True) #Removing all duplicates in the Retail Dataset
## Datatype mismatches 
df2.dtypes
# InvoiceNo, StockCode, description, country and customer ID should be in string formamt
df2["InvoiceNo"] = df2["InvoiceNo"].astype(str)
df2["StockCode"] = df2["StockCode"].astype(str)
df2["Description"] = df2["Description"].astype(str)
df2["CustomerID"] = df2["CustomerID"].astype(str)
df2["Country"] = df2["Country"].astype(str)
## Invalid values - Quantity
df2.loc[df2['Quantity'] < 0, 'Quantity'] = np.nan #replaced negattive values with nan
for i in df2["Quantity"]:
    counter = 0
    if i < 0:
        counter += 1

print("Negative Quantity vals:",counter)


#QUANTITY VALS SHOULD NOT BE NEGATIVE. replaced all negative quantity with nan
## Checking inconsistent formating in columns
# pattern definitions
invoice_no_pattern = r'^(C)?\d{6}$'
stock_code_pattern = r'^\d{5}$'
description_pattern = r'^(?!\d). + '
quantity_pattern = r'^\d+$'
date_pattern = r'^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$'
unit_price_pattern = r'^\d+\.\d{2}$'
customer_id_pattern = r'^\d{5}$'
country_pattern = r'^[a-zA-Z\s]+$'

#inconsistencies in Invoice col
inconsistent_invoice = df2[~df2["InvoiceNo"].astype(str).str.match(invoice_no_pattern)]
inconsistent_invoice
#Inconsistencies in stock col
inconsistent_stock = df2[~df2["StockCode"].astype(str).str.match(stock_code_pattern)]
inconsistent_stock

#checkinng inconsistencies in descrp col
inconsistrnt_descrp = df2[~df2["Description"].astype(str).str.match(description_pattern)]

#checking inconsistencies in quantity col
inconsistent_quan = df2[~df2["Quantity"].astype(str).str.match(quantity_pattern)]

#chhecking inconsistencies in date col
inconsistent_date = df2[~df2["InvoiceDate"].astype(str).str.match(date_pattern)]

#checking inconsistencies in prices col
inconsistent_price = df2[~df2["UnitPrice"].astype(str).str.match(unit_price_pattern)]

#checking inconsistencies in customer col
inconsistent_custm = df2[~df2["CustomerID"].astype(str).str.match(customer_id_pattern)]

#checking inconsistencies in country col
inconsistent_ctry = df2[~df2["Country"].astype(str).str.match(country_pattern)]



## Handling the inconsistent formating in columns

#InvoiceNo - remove rows with inconsistency
# df3 = df3[df3["InvoiceNo"].as

#Stock code - remove rows with more than 5 digits i.e only 5 digits should represent a stock code
df2["StockCode"] = df2["StockCode"].str.extract("(\d{5})")

#removing special characters in descrp col
df2['Description'] = df2['Description'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

#Unit Price - convert to standardized rounding decimal places
df2["UnitPrice"] = df2["UnitPrice"].round(2)

# Country - country name sshould be in title case
df2["Country"] = df2["Country"].str.title()


df2.describe()
# Minimumm value for unit price is negative, so we ned to remove negative values as they are  not needed

df2 = df2[df2["UnitPrice"]>0]
df2.describe()
# Outliers -Quantity
# Numerical columnds are the quantity and unitprice cols, so i'll be using box plot viusalisation to show the outliers of each columns

fig, axes = plt.subplots(1, 2, figsize=(8, 4))
#Quantity: Most values are within the range of 0 to 10000,  and there are two outluiers around 70000 and 80000
axes[0].boxplot(df2["Quantity"])
axes[0].set_title("Quantity")
#UnitPrice: Values are within the range of 1 and 4000, 3 ooutliers present between 8000 and 14000
axes[1].boxplot(df2["UnitPrice"])
axes[1].set_title("UnitPrice")

## Cleaned data
df2.to_csv("cleaned_retail.csv", index=False)